# 1.Import dependencies

In [1]:
import cv2
import numpy as np

# 2. Import Models

In [15]:
import joblib

FC_MODEL = joblib.load("../../models/Face Detector/FC_F640_A87.pkl")

# 3. Define Helper Functions

In [16]:
VIDEO_CAPTURE_FRAME_SIZE = (640, 480)

FC_FRAME_SIZE = 640

FC_HOG_WIN_SIZE = (FC_FRAME_SIZE, FC_FRAME_SIZE) 
FC_HOG_BLOCK_SIZE = (32, 32)
FC_HOG_BLOCK_STRIDE = (32, 32)
FC_HOG_CELL_SIZE = (8, 8)
FC_HOG_NUM_BINS = 9


FC_HOG = cv2.HOGDescriptor(FC_HOG_WIN_SIZE, FC_HOG_BLOCK_SIZE, FC_HOG_BLOCK_STRIDE, FC_HOG_CELL_SIZE, FC_HOG_NUM_BINS)

def extract_features_for_fc(frame) -> list[list]:
    """
    returns the feature vector for the face classifier
    """
    gray_frame = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), FC_HOG_WIN_SIZE)
    hog = FC_HOG.compute(gray_frame).flatten() # gray_frame[64:577, 64:577] -> crops the centermost 512x512 window
    
    return [hog / hog.max()]

# 4. Main Application

In [17]:
camera = cv2.VideoCapture(0)

while camera.isOpened():

    ret, frame = camera.read()

    fc_features = extract_features_for_fc(np.copy(frame))

    face_in_frame = FC_MODEL.predict(fc_features)

    if face_in_frame:
        # Draw a green rectangle if a face is detected
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 255, 0), 5)
    else:
        # Draw a red rectangle if no face is detected
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 5)
    
    cv2.imshow("Testing", frame)

    if cv2.waitKey(25) & 0xFF == ord('q'): 
        print(frame.shape)
        break # Press 'q' to exit the loop

# Release the video capture object and close the OpenCV windows
camera.release()
cv2.destroyAllWindows()

(480, 640, 3)
